# ETL Transfer into a Database for long term index data

Load in the data from an Excel sheet drop the empty columns and rows with bad data

In [37]:
import pandas as pd

df_cape_ratio = pd.read_excel("data/ie_data.xls", sheet_name="Data", skiprows=7, index_col="Date")
df_cape_ratio = df_cape_ratio.drop(columns=['Unnamed: 13', 'Unnamed: 15'])
df_cape_ratio = df_cape_ratio.drop(df_cape_ratio.index[-1])
df_cape_ratio

,P,D,E,CPI,Fraction,Rate GS10,Price,Dividend,Price.1,Earnings,Earnings.1,CAPE,TR CAPE,Yield,Returns,Returns.1,Real Return,Real Return.1,Returns.2
Date,,,,,,,,,,,,,,,,,,,
1871.01,4.44,0.260000,0.4,12.464061,1871.041667,5.32,104.974369,6.147148,1.049744e+02,9.457150,9.457150,NaN,NaN,NaN,1.004177,1.000000,0.130609,0.092504,0.038106
1871.02,4.5,0.260000,0.4,12.844641,1871.125000,5.323333,103.240573,5.965011,1.037377e+02,9.176940,9.221125,NaN,NaN,NaN,1.004180,0.974424,0.130858,0.094635,0.036224
1871.03,4.61,0.260000,0.4,13.034972,1871.208333,5.326667,104.219911,5.877913,1.052139e+02,9.042942,9.129188,NaN,NaN,NaN,1.004183,0.964209,0.130951,0.096186,0.034765
1871.04,4.74,0.260000,0.4,12.559226,1871.291667,5.33,111.218062,6.100569,1.127920e+02,9.385490,9.518314,NaN,NaN,NaN,1.004185,1.004919,0.122056,0.090972,0.031084
1871.05,4.86,0.260000,0.4,12.273812,1871.375000,5.333333,116.685446,6.242431,1.188643e+02,9.603740,9.783073,NaN,NaN,NaN,1.004188,1.032591,0.122638,0.089488,0.033150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022.03,4391.265217,61.969974,197.91,287.504,2022.208333,2.13,4500.965133,63.518071,2.891616e+06,202.854067,130322.300006,34.270799,37.239412,0.030717,0.948016,47.304227,NaN,NaN,NaN
2022.04,4391.296,62.653316,NaN,289.109,2022.291667,2.75,4476.009224,63.861972,2.879003e+06,NaN,NaN,33.889165,36.800852,0.025106,0.989378,44.596200,NaN,NaN,NaN
2022.05,4040.36,63.336658,NaN,292.296,2022.375000,2.9,4073.400036,63.854594,2.623464e+06,NaN,NaN,30.803768,33.469559,0.027804,0.981988,43.641414,NaN,NaN,NaN


Now rename the columns, convert columns with numerical data from string to floats, convert the index to a string then to a datetime.

In [38]:
df_cape_ratio.columns = ["S&P_comp_p", "dividend", "earnings", "CPI", "date_fraction", "long_interest_rate_GS10",
                         "real_price", "real_dividend", "real_total_return_price", "real_earnings",
                         "real_tr_scaled_earnings","CAPE", "TR_CAPE", "Excess_CAPE_yield", "monthly_total_bond_returns",
                         "real_total_bond_returns",
                         "10_year_annualized_stock_real_return", "10_year_annualized_bond_real_return",
                         "10_year_excess_annualized_return"]

df_cape_ratio["S&P_comp_p"] = pd.to_numeric(df_cape_ratio["S&P_comp_p"])
df_cape_ratio["earnings"] = pd.to_numeric(df_cape_ratio["earnings"])
df_cape_ratio["CPI"] = pd.to_numeric(df_cape_ratio["CPI"])
df_cape_ratio["long_interest_rate_GS10"] = pd.to_numeric(df_cape_ratio["long_interest_rate_GS10"])

df_cape_ratio.index = df_cape_ratio.index.astype(str)
df_cape_ratio.index = pd.to_datetime(df_cape_ratio.index, format="%Y.%m")
df_cape_ratio.dtypes

S&P_comp_p                              float64
dividend                                float64
earnings                                float64
CPI                                     float64
date_fraction                           float64
long_interest_rate_GS10                 float64
real_price                              float64
real_dividend                           float64
real_total_return_price                 float64
real_earnings                           float64
real_tr_scaled_earnings                 float64
CAPE                                    float64
TR_CAPE                                 float64
Excess_CAPE_yield                       float64
monthly_total_bond_returns              float64
real_total_bond_returns                 float64
10_year_annualized_stock_real_return    float64
10_year_annualized_bond_real_return     float64
10_year_excess_annualized_return        float64
dtype: object

Now create a database if needed to upload the data to.

To create the database just uncomment the code.

In [39]:
from sqlalchemy import Table, Column, String, Integer, Float, ForeignKey, create_engine, MetaData, DateTime, insert, select, desc


engine = create_engine("sqlite:///capeDB.sqlite")

connection = engine.connect()
metadata = MetaData()

# sources = Table("CAPE_DATA", metadata,
#                 Column("id", Integer(), primary_key=True),
#                 Column("date", DateTime()),
#                 Column("S&P_comp_p", Float()),
#                 Column("dividend", Float()),
#                 Column("earnings", Float()),
#                 Column("CPI", Float()),
#                 Column("date_fraction", Float()),
#                 Column("long_interest_rate_GS10", Float()),
#                 Column("real_price", Float()),
#                 Column("real_dividend", Float()),
#                 Column("real_total_return_price", Float()),
#                 Column("real_earnings", Float()),
#                 Column("real_tr_scaled_earnings", Float()),
#                 Column("CAPE", Float()),
#                 Column("TR_CAPE", Float()),
#                 Column("Excess_CAPE_yield", Float()),
#                 Column("monthly_total_bond_returns", Float()),
#                 Column("real_total_bond_returns", Float()),
#                 Column("10_year_annualized_stock_real_return", Float()),
#                 Column("10_year_annualized_bond_real_return", Float()),
#                 Column("10_year_excess_annualized_return", Float()))
#
#
# metadata.create_all(engine)


Push the data to the database.

In [40]:
df_cape_ratio.to_sql("CAPE_DATA", con=engine, if_exists='append')

1819

Pull the data from the database and show it.

In [41]:
df_db = pd.read_sql("CAPE_DATA", con=engine, index_col="date")
df_db

,id,S&P_comp_p,dividend,earnings,CPI,date_fraction,long_interest_rate_GS10,real_price,real_dividend,real_total_return_price,real_earnings,real_tr_scaled_earnings,CAPE,TR_CAPE,Excess_CAPE_yield,monthly_total_bond_returns,real_total_bond_returns,10_year_annualized_stock_real_return,10_year_annualized_bond_real_return,10_year_excess_annualized_return
date,,,,,,,,,,,,,,,,,,,,
1871-01-01,1,4.440000,0.260000,0.40,12.464061,1871.041667,5.320000,104.974369,6.147148,1.049744e+02,9.457150,9.457150,NaN,NaN,NaN,1.004177,1.000000,0.130609,0.092504,0.038106
1871-02-01,2,4.500000,0.260000,0.40,12.844641,1871.125000,5.323333,103.240573,5.965011,1.037377e+02,9.176940,9.221125,NaN,NaN,NaN,1.004180,0.974424,0.130858,0.094635,0.036224
1871-03-01,3,4.610000,0.260000,0.40,13.034972,1871.208333,5.326667,104.219911,5.877913,1.052139e+02,9.042942,9.129188,NaN,NaN,NaN,1.004183,0.964209,0.130951,0.096186,0.034765
1871-04-01,4,4.740000,0.260000,0.40,12.559226,1871.291667,5.330000,111.218062,6.100569,1.127920e+02,9.385490,9.518314,NaN,NaN,NaN,1.004185,1.004919,0.122056,0.090972,0.031084
1871-05-01,5,4.860000,0.260000,0.40,12.273812,1871.375000,5.333333,116.685446,6.242431,1.188643e+02,9.603740,9.783073,NaN,NaN,NaN,1.004188,1.032591,0.122638,0.089488,0.033150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-01,1815,4391.265217,61.969974,197.91,287.504000,2022.208333,2.130000,4500.965133,63.518071,2.891616e+06,202.854067,130322.300006,34.270799,37.239412,0.030717,0.948016,47.304227,NaN,NaN,NaN
2022-04-01,1816,4391.296000,62.653316,NaN,289.109000,2022.291667,2.750000,4476.009224,63.861972,2.879003e+06,NaN,NaN,33.889165,36.800852,0.025106,0.989378,44.596200,NaN,NaN,NaN
2022-05-01,1817,4040.360000,63.336658,NaN,292.296000,2022.375000,2.900000,4073.400036,63.854594,2.623464e+06,NaN,NaN,30.803768,33.469559,0.027804,0.981988,43.641414,NaN,NaN,NaN
